In [3]:
%pip install scikeras


  Using cached scikeras-0.10.0-py3-none-any.whl.metadata (4.1 kB)
Using cached scikeras-0.10.0-py3-none-any.whl (27 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf

from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

In [5]:
data=pd.read_csv('Churn_Modelling.csv')


In [6]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
#Encode one categorical variables-Gender
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
#One hot encoding 'Geography'
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoded=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [7]:
X = data.drop('Exited', axis=1)
y = data['Exited']

In [8]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
import pickle
## Save the encoders and scaler
with open('label_enocoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)


In [11]:
#Defining a funtion to create a model with trying differnt parameters(Keras Classifier)
def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))
    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model

In [20]:
## Create a keras classifier
model=KerasClassifier(build_fn=create_model,verbose=1,neurons=32,layers=1)


In [21]:
#Define grid search params
param_grid={
    'neurons':[16,32,64,128],
    'layers':[1,2],
    'epochs':[50,100]

}

In [23]:
#perofrm grid search
grid=GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=1,cv=3)
grid_result=grid.fit(X_train,y_train)
#print the best para
print("best:%f using %s" % (grid_result.best_score_,grid_result.best_params_))

Epoch 1/50


c:\Users\satya\Documents\data_science_roadmap\dl_projects\ANN_Classification\venv\lib\site-packages\scikeras\wrappers.py:302: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"


167/167 [==============================] - 1s 1ms/step - loss: 0.6379 - accuracy: 0.6374
Epoch 2/50
167/167 [==============================] - 0s 1ms/step - loss: 0.4635 - accuracy: 0.8026
Epoch 3/50
167/167 [==============================] - 0s 1ms/step - loss: 0.4366 - accuracy: 0.8132
Epoch 4/50
167/167 [==============================] - 0s 1ms/step - loss: 0.4254 - accuracy: 0.8170
Epoch 5/50
167/167 [==============================] - 0s 1ms/step - loss: 0.4173 - accuracy: 0.8207
Epoch 6/50
167/167 [==============================] - 0s 988us/step - loss: 0.4093 - accuracy: 0.8266
Epoch 7/50
167/167 [==============================] - 0s 941us/step - loss: 0.4013 - accuracy: 0.8324
Epoch 8/50
167/167 [==============================] - 0s 904us/step - loss: 0.3930 - accuracy: 0.8350
Epoch 9/50
167/167 [==============================] - 0s 942us/step - loss: 0.3858 - accuracy: 0.8402
Epoch 10/50
167/167 [==============================] - 0s 963us/step - loss: 0.3789 - accuracy: 0.8442


c:\Users\satya\Documents\data_science_roadmap\dl_projects\ANN_Classification\venv\lib\site-packages\scikeras\wrappers.py:302: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"


167/167 [==============================] - 1s 1000us/step - loss: 0.5972 - accuracy: 0.7238
Epoch 2/50
167/167 [==============================] - 0s 981us/step - loss: 0.4851 - accuracy: 0.7965
Epoch 3/50
167/167 [==============================] - 0s 995us/step - loss: 0.4478 - accuracy: 0.8039
Epoch 4/50
167/167 [==============================] - 0s 976us/step - loss: 0.4309 - accuracy: 0.8162
Epoch 5/50
167/167 [==============================] - 0s 976us/step - loss: 0.4213 - accuracy: 0.8217
Epoch 6/50
167/167 [==============================] - 0s 991us/step - loss: 0.4132 - accuracy: 0.8266
Epoch 7/50
167/167 [==============================] - 0s 983us/step - loss: 0.4057 - accuracy: 0.8329
Epoch 8/50
167/167 [==============================] - 0s 972us/step - loss: 0.3980 - accuracy: 0.8401
Epoch 9/50
167/167 [==============================] - 0s 988us/step - loss: 0.3895 - accuracy: 0.8453
Epoch 10/50
167/167 [==============================] - 0s 1ms/step - loss: 0.3810 - accuracy

...

167/167 [==============================] - 1s 2ms/step - loss: 0.4432 - accuracy: 0.8132
Epoch 2/100
167/167 [==============================] - 0s 2ms/step - loss: 0.3845 - accuracy: 0.8440
Epoch 3/100
167/167 [==============================] - 0s 2ms/step - loss: 0.3544 - accuracy: 0.8556
Epoch 4/100
167/167 [==============================] - 0s 2ms/step - loss: 0.3434 - accuracy: 0.8599
Epoch 5/100
167/167 [==============================] - 0s 2ms/step - loss: 0.3358 - accuracy: 0.8639
Epoch 6/100
167/167 [==============================] - 0s 2ms/step - loss: 0.3306 - accuracy: 0.8644
Epoch 7/100
167/167 [==============================] - 0s 2ms/step - loss: 0.3259 - accuracy: 0.8646
Epoch 8/100
167/167 [==============================] - 0s 2ms/step - loss: 0.3199 - accuracy: 0.8657
Epoch 9/100
167/167 [==============================] - 0s 2ms/step - loss: 0.3179 - accuracy: 0.8672
Epoch 10/100
167/167 [==============================] - 0s 2ms/step - loss: 0.3135 - accuracy: 0.8712
E

c:\Users\satya\Documents\data_science_roadmap\dl_projects\ANN_Classification\venv\lib\site-packages\scikeras\wrappers.py:302: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"


167/167 [==============================] - 1s 2ms/step - loss: 0.4384 - accuracy: 0.8131
Epoch 2/100
167/167 [==============================] - 0s 2ms/step - loss: 0.3757 - accuracy: 0.8451
Epoch 3/100
167/167 [==============================] - 0s 2ms/step - loss: 0.3453 - accuracy: 0.8624
Epoch 4/100
167/167 [==============================] - 0s 2ms/step - loss: 0.3364 - accuracy: 0.8633
Epoch 5/100
167/167 [==============================] - 0s 2ms/step - loss: 0.3306 - accuracy: 0.8630
Epoch 6/100
167/167 [==============================] - 0s 3ms/step - loss: 0.3263 - accuracy: 0.8650
Epoch 7/100
167/167 [==============================] - 0s 2ms/step - loss: 0.3220 - accuracy: 0.8665
Epoch 8/100
167/167 [==============================] - 0s 2ms/step - loss: 0.3159 - accuracy: 0.8690
Epoch 9/100
167/167 [==============================] - 0s 2ms/step - loss: 0.3147 - accuracy: 0.8706
Epoch 10/100
167/167 [==============================] - 0s 2ms/step - loss: 0.3082 - accuracy: 0.8718
E

c:\Users\satya\Documents\data_science_roadmap\dl_projects\ANN_Classification\venv\lib\site-packages\scikeras\wrappers.py:302: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"


250/250 [==============================] - 1s 1ms/step - loss: 0.5259 - accuracy: 0.7613
Epoch 2/50
250/250 [==============================] - 0s 2ms/step - loss: 0.4477 - accuracy: 0.8104
Epoch 3/50
250/250 [==============================] - 0s 1ms/step - loss: 0.4316 - accuracy: 0.8163
Epoch 4/50
250/250 [==============================] - 0s 1ms/step - loss: 0.4213 - accuracy: 0.8238
Epoch 5/50
250/250 [==============================] - 0s 1ms/step - loss: 0.4111 - accuracy: 0.8279
Epoch 6/50
250/250 [==============================] - 0s 2ms/step - loss: 0.4011 - accuracy: 0.8335
Epoch 7/50
250/250 [==============================] - 0s 2ms/step - loss: 0.3907 - accuracy: 0.8394
Epoch 8/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3809 - accuracy: 0.8436
Epoch 9/50
250/250 [==============================] - 0s 2ms/step - loss: 0.3728 - accuracy: 0.8456
Epoch 10/50
250/250 [==============================] - 0s 2ms/step - loss: 0.3658 - accuracy: 0.8491
Epoch 11/5